In [32]:
import requests
from requests.auth import HTTPBasicAuth
from flask import Flask, request
import threading
import webbrowser
import time
import pandas as pd

# Variáveis Globais
app = Flask(__name__)
authorization_code = None

# Configurações OAuth
client_id = '419759cc09659588aa42c22986968016e4ce2adc'
client_secret = 'ab1097471db5ec489f4e285b35098d95bb27c4469eb6abeabb99fecf46ea'
redirect_url = 'http://localhost:5000/callback'
state = "8bde85dd6e729dcd6e0d01dde003469d"
authorization_url = f'https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_url}&state={state}'
token_url = 'https://www.bling.com.br/Api/v3/oauth/token'
url_base = 'https://www.bling.com.br/Api/v3'
max_pages = 2   # None para todas as páginas
page_limit = 500
id_deposito_escritorio = 863558208

@app.route('/callback')
def callback():
    global authorization_code
    authorization_code = request.args.get('code')
    return 'Authorization code recebido. Você pode fechar esta janela.'

def start_flask_app(start_event):
    start_event.set()
    app.run(port=5000)

def get_authorization_code(client_id: str, redirect_uri: str, state: str) -> str:
    """
    Inicia o processo de obtenção de um código de autorização OAuth2 do Bling.

    Esta função inicia um servidor Flask em uma thread separada para lidar com o callback de autorização,
    constrói uma URL de autorização e abre-a no navegador padrão do usuário. Em seguida, aguarda até que
    o código de autorização seja recebido e definido.

    Parâmetros:
    client_id (str): ID do cliente que está fazendo a requisição.
    redirect_uri (str): URI para onde o usuário será redirecionado após a autorização.
    state (str): Parâmetro opcional para manter o estado entre a requisição e o callback.

    Retorna:
    str: O código de autorização obtido após o usuário autorizar a aplicação.

    Exceções:
    Esta função assume que há uma variável global `authorization_code` que será definida pela aplicação Flask.
    Certifique-se de que o servidor Flask está corretamente configurado para definir essa variável.

    Exemplo de uso:
        client_id = "meu_cliente_id"
        redirect_uri = "https://minhaapp.com/callback"
        state = "estado_arbitrario"

        authorization_code = get_authorization_code(client_id, redirect_uri, state)
        print(f"O código de autorização é: {authorization_code}")
    """
    start_event = threading.Event()
    threading.Thread(target=start_flask_app, args=(start_event,)).start()
    start_event.wait()  # Espera até que o servidor Flask esteja pronto

    url = f'https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&state={state}'
    webbrowser.open(url)

    # Aguarda até que o `authorization_code` seja definido
    while not authorization_code:
        time.sleep(1)
    
    return authorization_code

def get_access_token(client_id: str, client_secret: str, authorization_code: str) -> dict:
    """
    Obtém um token de acesso OAuth2 do Bling usando um código de autorização.

    Esta função envia uma requisição POST para o endpoint de token do Bling,
    trocando o código de autorização por um token de acesso.

    Parâmetros:
    client_id (str): ID do cliente que está fazendo a requisição.
    client_secret (str): Segredo do cliente usado para autenticação.
    authorization_code (str): Código de autorização obtido após o usuário autorizar a aplicação.

    Retorna:
    dict: A resposta JSON contendo o token de acesso e outros dados relacionados à autenticação.

    Exemplo de uso:
        client_id = "meu_cliente_id"
        client_secret = "meu_segredo_do_cliente"
        authorization_code = "meu_codigo_de_autorizacao"

        access_token_response = get_access_token(client_id, client_secret, authorization_code)
        print(f"O token de acesso é: {access_token_response['access_token']}")
    """
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '1.0'
    }
    body = {
        'grant_type': 'authorization_code',
        'code': authorization_code,
        'redirect_uri': 'http://localhost:5000/callback'
    }
    response = requests.post(token_url, headers=headers, data=body, auth=HTTPBasicAuth(client_id, client_secret))
    return response.json()

# Obter o authorization_code
authorization_code = get_authorization_code(client_id, redirect_url, state)

# Obter o access_token usando o authorization_code
token_response = get_access_token(client_id, client_secret, authorization_code)
access_token = token_response.get('access_token')

print(access_token)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


00fb4e81864ad30f02587425dd0c5ee913987443


In [33]:
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

tabela_final = []
pagina = 219
while True:
    url = f'https://www.bling.com.br/Api/v3/produtos?pagina={pagina}&limite=500&criterio=1&tipo=T'
    requisicao = requests.get(url, headers=headers)
    informacao = requisicao.json()
    tabela = informacao.get('data', [])
    if len(informacao['data']) < 1:
        break
    prod_df = pd.DataFrame(tabela)
    tabela_final.append(prod_df)
    pagina += 1

df_produtos = pd.concat(tabela_final, ignore_index=True)

df_produtos

,id,idProdutoPai,nome,codigo,preco,tipo,situacao,formato,descricaoCurta,imagemURL
0,1794985347,1.794985e+09,Polo Guess Masculina Piquet Preta Tamanho:S/P,20178,199.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",https://easyreturn.com.br/modulos/imgprodutos/...
1,1794985257,NaN,Polo Guess Masculina Piquet Preta,20177,199.99,P,A,V,"<h2 style=""box-sizing: border-box; font-family...",https://easyreturn.com.br/modulos/imgprodutos/...
2,1790878551,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20167,329.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",https://easyreturn.com.br/modulos/imgprodutos/...
3,1790878545,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20166,329.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",https://easyreturn.com.br/modulos/imgprodutos/...
4,1790878539,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20165,329.99,P,A,S,"<h2 style=""box-sizing: border-box; font-family...",https://easyreturn.com.br/modulos/imgprodutos/...
...,...,...,...,...,...,...,...,...,...,...
365,1164833512,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1038,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
366,1164833499,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1037,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
367,1164833485,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1036,189.99,P,A,S,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...
368,1164833412,NaN,Camiseta Ralph Lauren Custom Fit Chumbo,1036-1,189.99,P,A,V,"<h2 style=""box-sizing: inherit; margin: 0px 0p...",https://easyreturn.com.br/modulos/imgprodutos/...


In [34]:
# Transformando em um DF e removendo coluna de descrição
lista_produtos = pd.DataFrame(df_produtos)
lista_produtos.drop(columns=['descricaoCurta'], inplace=True)
lista_produtos

,id,idProdutoPai,nome,codigo,preco,tipo,situacao,formato,imagemURL
0,1794985347,1.794985e+09,Polo Guess Masculina Piquet Preta Tamanho:S/P,20178,199.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
1,1794985257,NaN,Polo Guess Masculina Piquet Preta,20177,199.99,P,A,V,https://easyreturn.com.br/modulos/imgprodutos/...
2,1790878551,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20167,329.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
3,1790878545,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20166,329.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
4,1790878539,1.790879e+09,Polo Ralph Lauren Masculina Custom Fit Coloure...,20165,329.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
...,...,...,...,...,...,...,...,...,...
365,1164833512,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1038,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
366,1164833499,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1037,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
367,1164833485,1.164833e+09,Camiseta Ralph Lauren Custom Fit Chumbo Tamanh...,1036,189.99,P,A,S,https://easyreturn.com.br/modulos/imgprodutos/...
368,1164833412,NaN,Camiseta Ralph Lauren Custom Fit Chumbo,1036-1,189.99,P,A,V,https://easyreturn.com.br/modulos/imgprodutos/...


In [35]:
id_produto = lista_produtos['id'][0]
codigo_produto = str(lista_produtos['codigo'][0])
print(id_produto, codigo_produto)
len(lista_produtos)

1794985347 20178


370

In [38]:
def consultar_estoque(id_produto, codigo_produto):
    # Configuração da API
    base_url = "https://www.bling.com.br/Api/v3/estoques/saldos"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    url_estoque = f'{base_url}?idsProdutos[]={id_produto}&codigo={codigo_produto}'
    requisicao = requests.get(url_estoque, headers=headers)
    if requisicao.status_code != 200:
        print(f'Erro na requisição: {requisicao.status_code}')
        return None

    informacao = requisicao.json()
    tabela = informacao.get('data', [])
    if not tabela:
        print('Nenhuma informação encontrada para o produto especificado.')
        return None

    produto_info = tabela[0].get('produto', {})
    saldo_fisico = tabela[0].get('saldoFisicoTotal', 'N/A')
    saldo_virtual = tabela[0].get('saldoVirtualTotal', 'N/A')
    depositos_info = tabela[0].get('depositos', [])

    # Criar um DF apenas com a primeira linha dos dados
    deposito_info_0 = [depositos_info[0]]  # Pega somente a primeira linha
    estoque_df = pd.DataFrame(deposito_info_0)

    estoque_df['id_produto'] = id_produto
    estoque_df['codigo'] = codigo_produto
    estoque_df['saldo'] = saldo_virtual

    return estoque_df


tabela_estoque = []
i = 0
while i != 370:
    info = consultar_estoque(lista_produtos['id'][i], codigo_produto = lista_produtos['codigo'][i])
    df_estoque = pd.DataFrame(info)
    tabela_estoque.append(df_estoque)
    i += 1

df_estoque = pd.concat(tabela_estoque, ignore_index=True)
display(df_estoque)


In [ ]:
tabela_estoque = []
i = 0
for i in len(lista_produtos):
    id_produto = lista_produtos['id'][i]
    codigo_produto = lista_produtos['codigo'][i]
    # Adicionando parametros ao URL
    url_estoque = f'{url_base}/estoques/saldos?idsProdutos[]={id_produto}&codigo={codigo_produto}&'
    requisicao = requests.get(url, headers=headers)
    if requisicao.status_code == 200:
        data = requisicao.json()
    # Extraindo os dados de saldo
    saldo_info = data.get('data', [])
    estoque_df = pd.DataFrame(saldo_info) 
    tabela_estoque.append(estoque_df)
    i += 1

df_estoque = pd.concat(tabela_estoque, ignore_index=True)
print(df_estoque)

In [ ]:
tabela_estoque = []
i = 0
for i in len(lista_produtos):
    id_produto = lista_produtos['id'][i]
    codigo_produto = lista_produtos['codigo'][i]
    # Adicionando parametros ao URL
    url_estoque = f'{url_base}/estoques/saldos?idsProdutos[]={id_produto}&codigo={codigo_produto}&'
    requisicao = requests.get(url, headers=headers)
    if requisicao.status_code == 200:
        data = requisicao.json()
    # Extraindo os dados de saldo
    saldo_info = data.get('data', [])
    if saldo_info:
        produto_info = saldo_info[0].get('produto', {})
        saldo_fisico_total = saldo_info[0].get('saldoFisicoTotal', 'N/A')
        saldo_virtual_total = saldo_info[0].get('saldoVirtualTotal', 'N/A')
        depositos_info = saldo_info[0].get('depositos', [])

        # Criando um DataFrame com as informações do saldo
        df = pd.DataFrame(depositos_info)
        df['produto_id'] = produto_info.get('id', 'N/A')
        df['produto_codigo'] = codigo_produto
        df['saldo_fisico_total'] = saldo_fisico_total
        df['saldo_virtual_total'] = saldo_virtual_total
        df.rename(columns={
            'id': 'deposito_id',
            'saldoFisico': 'saldo_fisico',
            'saldoVirtual': 'saldo_virtual'
        }, inplace=True)
    else:
        print("Nenhum saldo encontrado para o produto.")
      tabela_estoque = data.get('data', []) 

df_estoque = pd.concat(tabela_estoque, ignore_index=True)
print(df_estoque)